# This is a simple tcp server test :

In [ ]:
import socket
import threading
import time
import json, types,string

class ClientData():
    clientsocket=''
    
    
    def __init__(self,client):
        self.clientsocket = client
        

class SocketServer(socket.socket):
    server_name='Rem'
    clients = []
    message = "this is a message."
#     =====================
#     HERE IS SERVER INIT
#     =====================    
    def __init__(self):
        socket.socket.__init__(self)
        #To silence- address occupied!!
        self.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.bind(('0.0.0.0', 5566))
        self.listen(5)

    def run(self):
        print ("Server started")
        try:
            self.accept_clients()
        except Exception as ex:
            print (ex)
        finally:
            print ("Server closed")
            for client in self.clients:
                client.close()
            self.close()
    def decode_data(self,data):
        splited = data.decode("utf-8").split(',')
        return splited 
#     =====================
#     HERE IS TEST FUNCTION
#     =====================
    
    def sendTime(self, client,jdata):
        import locale
        locale.setlocale(locale.LC_CTYPE, 'chinese')
        dict_hour={0:'半夜12點',1:'半夜1點',2:'半夜2點',3:'半夜3點',4:'半夜4點',5:'凌晨5點',6:'凌晨6點',7:'早上7點',
        8:'早上8點',7:'早上9點',8:'早上10點',9:'中午11點',10:'中午12點',11:'中午1點',12:'下午2點',13:'下午3點',
        14:'下午4點',15:'下午5點',16:'晚上6點',17:'晚上7點',18:'晚上8點',19:'晚上9點',20:'晚上10點',21:'晚上11點',}
        year, month, day, hour, minute,second = time.strftime("%Y,%m,%d,%H,%M,%S").split(',')

        time_str="現在是"+dict_hour[int(hour)]+"%s分%s秒呦" %(minute,second,)
        msg = {'cmd':'say','name':self.server_name, 'data':time_str}
        jmsg = json.dumps(msg)
        client.send(str.encode(jmsg))
    
    def greed(self,client,jdata):
        msg = {'cmd':'greed','name':self.server_name, 'data':'wtf'}
        jmsg = json.dumps(msg)
        client.send(str.encode(jmsg))
    
    def someoneSay(self,client,jdata):
        client_name=jdata['name']
        client_msg=jdata['data']
        msg =  {'cmd':'say','name':client_name, 'data':client_msg}
        jmsg=json.dumps(msg)
        print('jmsg=',jmsg,"!!")
        self.sendAll(jmsg)
        
    def sendAll(self,string):
        for client in self.clients:
            try :
                client.clientsocket.send(str.encode(string));
            except Exception as ex:
                print (ex)
                self.disconnect(client)
            finally:
                pass
            
        
        
#     WHEN CLIENT DISCONNECT
    def disconnect(self,client):
        #Removing client from clients list
        self.clients.remove(client)
        #Client Disconnected
        msg = {'cmd':'disconnect accept','name':self.server_name, 'data':''}
        jmsg = json.dumps(msg)
        client.send(str.encode(jmsg))
        self.onclose(client)
        #Closing connection with client
        client.clientsocket.close()


    dictCmdFunc= {'what time':sendTime,
              'my name is':greed,
              'say':someoneSay}
    def recieve(self, client):
        while 1:
            data = client.clientsocket.recv(1024)
            #Message Received
            self.onmessage(client, data)
            jdata = json.loads(data)
            print(jdata['cmd'])
            recv_cmd = jdata['cmd']
            
            if recv_cmd == 'disconnect':
                break
            else:                
                #Start thread 
                threading.Thread(target=self.dictCmdFunc[recv_cmd],args=(self,client.clientsocket,jdata,)).start()

        self.disconnect(client)
    
    def accept_clients(self):
        while 1:
            print("waiting for Client")
            (clientsocket, address) = self.accept()
            
            new_client = ClientData(clientsocket)
            #Adding client to clients list
            self.clients.append(new_client)
            #Client Connected
            self.onopen(clientsocket)
            #Start listening
            threading.Thread(target=self.recieve,args=(new_client,)).start()

    def broadcast(self, message):
        #Sending message to all clients
        for client in self.clients:
            client.clientsocket.send(message)
    # ===================
    # SOME EVENT FUNCTION
    # ===================
    def onopen(self, client):
        print ("Client Connected \n",client)

    def onmessage(self, client, message):
        print ("Client send message ",message)
        
    def onclose(self, client):
        print ("Client Disconnected")

def main():
    server = SocketServer()
    server.run()

if __name__ == "__main__":
    main()

Server started
waiting for Client
Client Connected 
 <socket.socket fd=1512, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 5566), raddr=('127.0.0.1', 59047)>
waiting for Client
Client send message  b'{"cmd":"say","name":"pohong","data":"123"}'jmsg=
 say{"cmd": "say", "name": "pohong", "data": "123"}
 !!
Client send message  b'{"cmd":"what time","name":"pohong","data":""}'
what time
Client send message  b'{"cmd":"what time","name":"pohong","data":""}'
what time
Client send message  b'{"cmd":"what time","name":"pohong","data":""}'
what time
Client send message  b'{"cmd":"what time","name":"pohong","data":""}'
what time
Client send message  b'{"cmd":"what time","name":"pohong","data":""}'
what time
Client send message  b'{"cmd":"what time","name":"pohong","data":""}'
what time
Client send message  b'{"cmd":"what time","name":"pohong","data":""}'
what time
Client send message  b'{"cmd":"what time","name":"pohong","data":""}'
what time
Client send m